In [1]:
import matplotlib.pyplot as plt
import numpy as np
from gym.wrappers import (
    FilterObservation,
    FlattenObservation,
    RecordVideo,
    RescaleAction,
    TimeLimit,
)
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.env_util import unwrap_wrapper

from ea_train import ARESEACheetah


initializing ocelot...


In [2]:
env = ARESEACheetah(
    incoming_mode="random",
    incoming_values=None,
    misalignment_mode="random",
    misalignment_values=None,
    action_mode="direct",
    magnet_init_mode="random",
    magnet_init_values=None,
    reward_mode="differential",
    target_beam_mode="random",
    target_beam_values=None,
    w_mu_x=1.0,
    w_mu_y=1.0,
    w_on_screen=1.0,
    w_sigma_x=1.0,
    w_sigma_y=1.0,
    w_time=1.0,
)
env = RescaleAction(env, -1, 1)

check_env(env)


done = False
done = False
done_reward = 0
self.w_done = 1.0
(done_reward_component) = 0.0


In [3]:
env.reset()

{'beam': array([-0.02273769,  0.0014518 , -0.10491275,  0.00418624], dtype=float32),
 'magnets': array([ 5.9285854e+01,  6.0291634e+01, -6.1429283e-03,  8.9177351e+00,
        -6.0066357e-03], dtype=float32),
 'target': array([-0.00049748,  0.00165402, -0.00157414,  0.00032111], dtype=float32),
 'incoming': array([ 1.5408890e+08, -3.9819602e-04,  1.6712205e-05,  4.4333999e-04,
         8.7518652e-05,  1.9711920e-04,  4.3197695e-05,  4.0057032e-05,
         2.0436040e-05,  4.8125894e-05,  6.2578759e-04], dtype=float32),
 'misalignments': array([ 0.0019492 ,  0.00147858,  0.00097263,  0.00117276,  0.00010727,
         0.00092272, -0.00191018,  0.00029119], dtype=float32)}

In [6]:
env.step(np.zeros(5))

done = False
done = False
done_reward = 0
self.w_done = 1.0
(done_reward_component) = 0.0


({'beam': array([1.5461736e-03, 2.1602718e-04, 3.3121699e-04, 5.7904133e-05],
        dtype=float32),
  'magnets': array([0., 0., 0., 0., 0.], dtype=float32),
  'target': array([-0.00049748,  0.00165402, -0.00157414,  0.00032111], dtype=float32),
  'incoming': array([ 1.5408890e+08, -3.9819602e-04,  1.6712205e-05,  4.4333999e-04,
          8.7518652e-05,  1.9711920e-04,  4.3197695e-05,  4.0057032e-05,
          2.0436040e-05,  4.8125894e-05,  6.2578759e-04], dtype=float32),
  'misalignments': array([ 0.0019492 ,  0.00147858,  0.00097263,  0.00117276,  0.00010727,
          0.00092272, -0.00191018,  0.00029119], dtype=float32)},
 0.0,
 False,
 {'binning': array(4),
  'l1_distance': 0.005650210270687239,
  'on_screen_reward': 1,
  'pixel_size': array([1.32792e-05, 9.78760e-06]),
  'screen_resolution': array([612., 510.]),
  'time_reward': -1})